In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "float"
impute_method = "itbagdt"
fs_method, fs_ratio = "fcls", 0.5
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

/root/miniconda3/envs/kvb/lib/python3.10/site-packages/sklearn/impute/_iterative.py:801: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9169 | AUPR: 0.6534 | Acc: 95.32%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9203 | AUPR: 0.6619 | Acc: 95.35%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9221 | AUPR: 0.6679 | Acc: 95.42%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9166 | AUPR: 0.6553 | Acc: 95.39%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9195 | AUPR: 0.6652 | Acc: 95.44%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9215 | AUPR: 0.6724 | Acc: 95.42%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9221 | AUPR: 0.6679 | Acc: 95.42%
Internal test: AUC: 0.9306 | AUPR: 0.6998 | Acc: 95.18%
External test: AUC: 0.9210 | AUPR: 0.6595 | Acc: 95.02%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9161 | AUPR: 0.6383 | Acc: 95.10%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9163 | AUPR: 0.6363 | Acc: 95.14%
model(kernel=linear, C=1)
Valid:         AUC: 0.9155 | AUPR: 0.6343 | Acc: 95.15%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9163 | AUPR: 0.6363 | Acc: 95.14%
Internal test: AUC: 0.9250 | AUPR: 0.6795 | Acc: 95.18%
External test: AUC: 0.9158 | AUPR: 0.6190 | Acc: 94.70%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7722 | AUPR: 0.4527 | Acc: 94.94%
model(n_neighbors=5)
Valid:         AUC: 0.7949 | AUPR: 0.4954 | Acc: 95.17%
model(n_neighbors=7)
Valid:         AUC: 0.8107 | AUPR: 0.5224 | Acc: 95.31%
best model(n_neighbors=7)
Valid:         AUC: 0.8107 | AUPR: 0.5224 | Acc: 95.31%
Internal test: AUC: 0.8167 | AUPR: 0.5331 | Acc: 94.89%
External test: AUC: 0.8055 | AUPR: 0.5022 | Acc: 94.84%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8734 | AUPR: 0.4255 | Acc: 86.38%
best model()
Valid:         AUC: 0.8734 | AUPR: 0.4255 | Acc: 86.38%
Internal test: AUC: 0.8819 | AUPR: 0.4471 | Acc: 86.57%
External test: AUC: 0.8686 | AUPR: 0.4023 | Acc: 85.09%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9172 | AUPR: 0.6551 | Acc: 95.38%
model(C=10.0)
Valid:         AUC: 0.9170 | AUPR: 0.6549 | Acc: 95.37%
model(C=100.0)
Valid:         AUC: 0.9169 | AUPR: 0.6549 | Acc: 95.36%
best model(C=1.0)
Valid:         AUC: 0.9172 | AUPR: 0.6551 | Acc: 95.38%
Internal test: AUC: 0.9255 | AUPR: 0.6889 | Acc: 95.23%
External test: AUC: 0.9148 | AUPR: 0.6370 | Acc: 94.84%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8774 | AUPR: 0.5714 | Acc: 83.32%
model(criterion=log_loss)
Valid:         AUC: 0.8887 | AUPR: 0.5903 | Acc: 83.37%
model(criterion=entropy)
Valid:         AUC: 0.8887 | AUPR: 0.5903 | Acc: 83.37%
best model(criterion=log_loss)
Valid:         AUC: 0.8887 | AUPR: 0.5903 | Acc: 83.37%
Internal test: AUC: 0.8784 | AUPR: 0.6185 | Acc: 84.21%
External test: AUC: 0.8776 | AUPR: 0.5968 | Acc: 83.61%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9137 | AUPR: 0.6525 | Acc: 87.41%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9142 | AUPR: 0.6552 | Acc: 87.81%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9148 | AUPR: 0.6565 | Acc: 87.91%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9139 | AUPR: 0.6565 | Acc: 87.91%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9146 | AUPR: 0.6582 | Acc: 88.24%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9148 | AUPR: 0.6581 | Acc: 88.23%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9139 | AUPR: 0.6565 | Acc: 87.91%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9146 | AUPR: 0.6582 | Acc: 88.24%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9148 | AUPR: 0.6581 | Acc: 88.23%
best model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9148 | AUPR: 0.6565 | Acc: 8

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9178 | AUPR: 0.6581 | Acc: 95.26%
model(criterion=squared_error)
Valid:         AUC: 0.9178 | AUPR: 0.6581 | Acc: 95.29%
best model(criterion=friedman_mse)
Valid:         AUC: 0.9178 | AUPR: 0.6581 | Acc: 95.26%
Internal test: AUC: 0.9223 | AUPR: 0.6633 | Acc: 95.23%
External test: AUC: 0.9199 | AUPR: 0.6589 | Acc: 95.05%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9129 | AUPR: 0.6351 | Acc: 95.11%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9149 | AUPR: 0.6392 | Acc: 95.15%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9152 | AUPR: 0.6399 | Acc: 95.12%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9152 | AUPR: 0.6399 | Acc: 95.12%
Internal test: AUC: 0.9237 | AUPR: 0.6744 | Acc: 95.23%
External test: AUC: 0.9155 | AUPR: 0.6219 | Acc: 94.59%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)